In [7]:
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Dark2_5 as palette
from bokeh.layouts import column
import itertools
from read_file_table import get_multiindex_ctrl_file_table
import re

#Boilerplate code for bokeh to function
output_file("pileup_test.html")

#Declare path to file_table.csv and generate list of control accessions using function from read_file_table.py
file_table_path = "/Users/lij/PycharmProjects/eclip_analysis_pipeline/file_table.csv"
file_table_df = get_multiindex_ctrl_file_table(file_table_path)
file_table_df = file_table_df.sort_index()

def generate_pileup_figure(filepath, gene_name, cell):
    '''
    This function accepts a filepath to a pileup table .csv (ex. 28S_pileup_data.csv), 
    a gene name to be plotted, and the cell line used in the experiment.
    Outputs Bokeh plots.
    '''
    global file_table_df
    
    #Reads global variable file_table_df and pulls the control and eCLIP experiment accession numbers of the gene name
    filtered_file_table_df = file_table_df.loc[gene_name,cell].copy()
    ctrl_accession = filtered_file_table_df[filtered_file_table_df["experiment_type"] == "ctrl"]["experiment_accession"]
    data_accession = filtered_file_table_df[filtered_file_table_df["experiment_type"] == "eclip"]["experiment_accession"]
    
    #Make sure that there is only one accession number for the control and experiment of the gene name
    assert len(ctrl_accession) == 1, "Multiple accessions detected for controls"
    assert len(data_accession) == 1, "Multiple accessions detected for dataset"
    
    #Convert dataframe data into a string
    ctrl_accession = ctrl_accession[0]
    data_accession = data_accession[0]
    
    #Read pileup data .csv into a Pandas dataframe
    pileup_data = pd.read_csv(filepath)
    
    #Reset the index, adds 1 to it, and renames the "index" column into "nucleotide_position" for easier reference
    #Also sorts the columns in alphabetical order to make the Bokeh plots be displayed in order of "rep1" and "rep2", instead of random order
    pileup_data = pileup_data.reset_index()
    pileup_data.sort_index(axis=1,inplace=True)
    pileup_data.rename(columns={'index':'nucleotide_position'}, inplace=True)
    pileup_data['nucleotide_position'] = pileup_data['nucleotide_position']+1
    
    #Remove excess parts of each column name to make it easier to read
    pileup_data.columns = [column.replace("_sorted_CONTIG_","_") for column in pileup_data.columns]
    
    #Make pileup_data Bokeh-accessible
    source = ColumnDataSource(pileup_data)
    
    #Pull column names corresponding to the accessions of the input gene name
    ctrl_names = [column for column in pileup_data.columns if ctrl_accession in column]
    data_names = [column for column in pileup_data.columns if data_accession in column]
    
    #Get the reference contig name from the input file
    contig = ctrl_names[0].split("_")[-1].upper()
    
    assert len(ctrl_names) == 1, "More than one control dataset found"
    colors = itertools.cycle(palette)
    
    #The below code block generates the Bokeh plot.
    
    #CHANGE THESE DIMENSIONS TO CHANGE PLOT SIZE
    p = figure(plot_width=1000, plot_height=1000)
    p.title.text = f"{gene_name}, {contig} rRNA ({cell})"
    for ctrlset in ctrl_names:
        p.line(x="nucleotide_position", y=ctrlset,source=source,  color=next(colors),legend=(ctrl_accession+"_"+gene_name+"_CONTROL"))
    for dataset in data_names:
        p.line(x="nucleotide_position", y=dataset,source=source, line_width=2, color=next(colors),legend=dict(value=dataset))
    p.title.text_font_size = "24pt"
    p.xaxis.axis_label = "Nucleotide position"
    p.yaxis.axis_label = "Reads per Million"
    p.axis.axis_label_text_font_size = "24pt"
    p.axis.major_label_text_font_size = "16pt"
    p.legend.label_text_font_size = "16pt"
    p.legend.location='top_left'
    
    hover = HoverTool()
    hover.tooltips = "@nucleotide_position"

    hover.mode = 'vline'

    p.add_tools(hover)
    return p



#CHANGE THESE VARIABLES TO THE TARGET GENE AND CELL TYPE OF INTEREST
gene = 'DDX6'
cell_type = "K562"

#Paths to each rRNA contig pileup_table.csv dataset
path_28S = "/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/28s_pileup_table.csv"
path_18S = "/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/18s_pileup_table.csv"
path_58S = "/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/5.8s_pileup_table.csv"
path_5S = "/Users/lij/PycharmProjects/eclip_analysis_pipeline/20201027_pileup_data/pileup_output/5s_pileup_table.csv"

#Call Bokeh-figure generator for each contig
plot_28S = generate_pileup_figure(path_28S, gene, cell_type)
plot_18S = generate_pileup_figure(path_18S, gene, cell_type)
plot_58S = generate_pileup_figure(path_58S, gene, cell_type)
plot_5S = generate_pileup_figure(path_5S, gene, cell_type)

#Display all plots for each contig!
show(column(plot_28S,plot_18S, plot_58S, plot_5S))

(5070, 670)
(1869, 670)
(157, 670)
(121, 670)


In [61]:
#Generates alphabetical list of all 150 proteins in the eCLIP dataset.
test = "UCHL5, DROSHA, GTF2F1, UTP18, EIF4G2, PPIL4, AQR, PUS1, GNL3, TIA1, FASTKD2, WDR3, SUPV3L1, NKRF, QKI, DDX52, XRN2, LARP4, LIN28B, CPEB4, ZNF800, FTO, DDX3X, DDX42, XPO5, SRSF1, FUBP3, CDC40, SAFB, TAF15, HNRNPM, BCCIP, BCLAF1, SERBP1, HNRNPL, SF3A3, CSTF2, XRCC6, SLTM, PABPC4, FMR1, HNRNPC, LSM11, NSUN2, CSTF2T, RPS11, NIP7, IGF2BP3, DHX30, TBRG4, DDX55, HNRNPA1, BUD13, NONO, AATF, RBM5, PCBP1, RBM15, PRPF4, FKBP4, PUM2, TROVE2, SRSF9, U2AF2, CPSF6, AKAP8L, KHSRP, IGF2BP2, SFPQ, WDR43, DKC1, FXR2, ZC3H11A, ABCF1, RBFOX2, SDAD1, SSB, HNRNPU, ZC3H8, GPKOW, EIF3D, HNRNPUL1, SUGP2, NIPBL, NCBP2, SF3B4, DGCR8, EIF3H, PUM1, MATR3, SUB1, RBM22, NOLC1, PPIG, DDX24, FAM120A, YWHAG, TARDBP, NOL12, GEMIN5, STAU2, UPF1, AKAP1, HLTF, WRN, GRWD1, PCBP2, DDX59, PABPN1, SF3B1, SMNDC1, G3BP1, GRSF1, DDX6, MTPAP, PHF6, RPS3, EFTUD2, ILF3, SND1, DDX21, SLBP, POLR2G, UTP3, PTBP1, IGF2BP1, FXR1, SAFB2, SBDS, ZNF622, AGGF1, LARP7, PRPF8, FUS, TIAL1, APOBEC3C, AARS, EWSR1, KHDRBS1, TRA2A, ZRANB2, U2AF1, NPM1, HNRNPK, EXOSC5, EIF3G, DDX51, METAP2, YBX3, SRSF7"
test = test.split(', ')
print(sorted(test))

['AARS', 'AATF', 'ABCF1', 'AGGF1', 'AKAP1', 'AKAP8L', 'APOBEC3C', 'AQR', 'BCCIP', 'BCLAF1', 'BUD13', 'CDC40', 'CPEB4', 'CPSF6', 'CSTF2', 'CSTF2T', 'DDX21', 'DDX24', 'DDX3X', 'DDX42', 'DDX51', 'DDX52', 'DDX55', 'DDX59', 'DDX6', 'DGCR8', 'DHX30', 'DKC1', 'DROSHA', 'EFTUD2', 'EIF3D', 'EIF3G', 'EIF3H', 'EIF4G2', 'EWSR1', 'EXOSC5', 'FAM120A', 'FASTKD2', 'FKBP4', 'FMR1', 'FTO', 'FUBP3', 'FUS', 'FXR1', 'FXR2', 'G3BP1', 'GEMIN5', 'GNL3', 'GPKOW', 'GRSF1', 'GRWD1', 'GTF2F1', 'HLTF', 'HNRNPA1', 'HNRNPC', 'HNRNPK', 'HNRNPL', 'HNRNPM', 'HNRNPU', 'HNRNPUL1', 'IGF2BP1', 'IGF2BP2', 'IGF2BP3', 'ILF3', 'KHDRBS1', 'KHSRP', 'LARP4', 'LARP7', 'LIN28B', 'LSM11', 'MATR3', 'METAP2', 'MTPAP', 'NCBP2', 'NIP7', 'NIPBL', 'NKRF', 'NOL12', 'NOLC1', 'NONO', 'NPM1', 'NSUN2', 'PABPC4', 'PABPN1', 'PCBP1', 'PCBP2', 'PHF6', 'POLR2G', 'PPIG', 'PPIL4', 'PRPF4', 'PRPF8', 'PTBP1', 'PUM1', 'PUM2', 'PUS1', 'QKI', 'RBFOX2', 'RBM15', 'RBM22', 'RBM5', 'RPS11', 'RPS3', 'SAFB', 'SAFB2', 'SBDS', 'SDAD1', 'SERBP1', 'SF3A3', 'SF3B1',